<a href="https://colab.research.google.com/github/arhaz1407/-Recommendation-text-python-content-based/blob/main/Uji_Skripsi_OKE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalasi dependensi
!pip install sentence-transformers
!pip install seaborn

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
import pandas as pd
import numpy as np
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import matplotlib.pyplot as plt

# Pengaturan visualisasi
plt.style.use('ggplot')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.titlesize'] = 12
plt.rcParams['image.cmap'] = 'jet'
plt.rcParams['image.interpolation'] = 'none'
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['axes.grid'] = True
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.markersize'] = 8
colors = ['xkcd:pale orange', 'xkcd:sea blue', 'xkcd:pale red', 'xkcd:sage green', 'xkcd:terra cotta', 'xkcd:dull purple', 'xkcd:teal', 'xkcd: goldenrod', 'xkcd:cadet blue', 'xkcd:scarlet']

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Memuat data
file_path = 'Mapping_kategori_topik_AlQuran_Sheet2.csv'
data = pd.read_csv(file_path)


In [4]:
# Memastikan tidak ada nilai yang hilang
data = data.dropna()

In [5]:
# Fungsi untuk membersihkan teks
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Menghapus spasi ekstra
    text = re.sub(r'\([^)]*\)', '', text)  # Menghapus teks dalam tanda kurung
    return text.strip()

# Membersihkan teks dalam kolom 'terjemah'
data['clean_terjemah'] = data['terjemah'].apply(clean_text)

# Ekstraksi data teks
text_data = data['clean_terjemah'].tolist()

In [6]:
# Ekstraksi data teks
text_data = data['terjemah'].tolist()

In [7]:
# Memuat model Sentence Transformer
model = SentenceTransformer('cahya/distilbert-base-indonesian')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/273M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [8]:
# Membuat embedding
embeddings = model.encode(text_data, show_progress_bar=True)

Batches:   0%|          | 0/195 [00:00<?, ?it/s]

In [9]:
# Menghitung cosine similarity dari embedding
cos_sim_data = cosine_similarity(embeddings)

In [10]:
# Pendekatan berbasis string menggunakan CountVectorizer
count_vectorizer = CountVectorizer().fit_transform(data['terjemah'])
count_cosine_sim = cosine_similarity(count_vectorizer, count_vectorizer)

In [11]:
# Mengubah cosine similarity CountVectorizer menjadi DataFrame
count_cosine_sim_df = pd.DataFrame(count_cosine_sim, index=data.index, columns=data.index)

In [12]:
# Fungsi untuk mencari ayat yang memiliki redaksi yang sama atau mirip
def find_similar_ayat(query, data, n=5):
    results = data[data['terjemah'].str.contains(query, case=False, na=False)]
    return results.head(n)

In [13]:
# Fungsi rekomendasi
def give_recommendations(index, print_recommendation=False, print_genres=False):
    # Check if cos_sim_data is a DataFrame, if not, convert it
    if not isinstance(cos_sim_data, pd.DataFrame):
        cos_sim_data_df = pd.DataFrame(cos_sim_data)  # Convert to DataFrame
    else:
        cos_sim_data_df = cos_sim_data
    # Menggunakan cosine similarity dari Sentence Transformer
    index_recomm_transformer = cos_sim_data_df.loc[index].sort_values(ascending=False).index.tolist()[1:6]
    ayat_recomm_transformer = data['terjemah'].iloc[index_recomm_transformer].values

    # Menggunakan cosine similarity dari CountVectorizer
    index_recomm_count = count_cosine_sim_df.loc[index].sort_values(ascending=False).index.tolist()[1:6]
    ayat_recomm_count = data['terjemah'].iloc[index_recomm_count].values

    # Mencari ayat yang memiliki redaksi yang sama atau mirip
    query_ayat = data['terjemah'].iloc[index]
    similar_ayat = find_similar_ayat(query_ayat, data, n=5)

    if print_recommendation:
        print(f'The preferred ayat is: {data["terjemah"].iloc[index]} \n')
        print("\nRecommendations based on Sentence Transformer:")
        for k, ayat in enumerate(ayat_recomm_transformer, start=1):
            print(f'The number {k} recommended ayat: {ayat} \n')
        print("\nRecommendations based on CountVectorizer:")
        for k, ayat in enumerate(ayat_recomm_count, start=1):
            print(f'The number {k} recommended ayat: {ayat} \n')
        print("\nAdditional similar ayat based on string matching:")
        for k, ayat in enumerate(similar_ayat['terjemah'], start=1):
            print(f'The number {k} similar ayat: {ayat} \n')

    if print_genres:
        print(f'The surah of the preferred ayat is:\n {data["surah"].iloc[index]} \n')
        for k, q in enumerate(range(len(ayat_recomm_transformer)), start=1):
            plot_q = data['surah'].iloc[index_recomm_transformer[q]]
            print(f'The surah of the number {k} recommended ayat is this one:\n {plot_q} \n')

    return {
        'transformer': {'Ayat': ayat_recomm_transformer, 'Index': index_recomm_transformer},
        'count_vectorizer': {'Ayat': ayat_recomm_count, 'Index': index_recomm_count}
    }

In [14]:
give_recommendations(4917, True, True)

The preferred ayat is: Maka nikmat Tuhan kamu yang manakah yang kamu dustakan? 


Recommendations based on Sentence Transformer:
The number 1 recommended ayat: Maka nikmat Tuhan kamu yang manakah yang kamu dustakan? 

The number 2 recommended ayat: Maka nikmat Tuhan kamu yang manakah yang kamu dustakan? 

The number 3 recommended ayat: Maka nikmat Tuhan kamu yang manakah yang kamu dustakan? 

The number 4 recommended ayat: Maka nikmat Tuhan kamu yang manakah yang kamu dustakan? 

The number 5 recommended ayat: Maka nikmat Tuhan kamu yang manakah yang kamu dustakan? 


Recommendations based on CountVectorizer:
The number 1 recommended ayat: Tuhan langit dan bumi dan apa yang berada di antara keduanya dan Tuhan tempat-tempat terbit matahari. 

The number 2 recommended ayat: Tuhan Yang memelihara langit dan bumi dan apa yang ada di antara keduanya jika kamu adalah orang yang meyakini. 

The number 3 recommended ayat: Tidak ada Tuhan (yang berhak disembah) melainkan Dia Yang menghidupkan d

{'transformer': {'Ayat': array(['Maka nikmat Tuhan kamu yang manakah yang kamu dustakan?',
         'Maka nikmat Tuhan kamu yang manakah yang kamu dustakan?',
         'Maka nikmat Tuhan kamu yang manakah yang kamu dustakan?',
         'Maka nikmat Tuhan kamu yang manakah yang kamu dustakan?',
         'Maka nikmat Tuhan kamu yang manakah yang kamu dustakan?'],
        dtype=object),
  'Index': [4937, 4954, 4952, 4950, 4948]},
 'count_vectorizer': {'Ayat': array(['Tuhan langit dan bumi dan apa yang berada di antara keduanya dan Tuhan tempat-tempat terbit matahari.',
         'Tuhan Yang memelihara langit dan bumi dan apa yang ada di antara keduanya jika kamu adalah orang yang meyakini.',
         'Tidak ada Tuhan (yang berhak disembah) melainkan Dia Yang menghidupkan dan Yang mematikan. (Dialah) Tuhanmu dan Tuhan bapak-bapakmu yang terdahulu.',
         '(Tuhan) Yang Maha Pemurah',
         'Malaikat-malaikat dan Jibril naik (menghadap) kepada Tuhan dalam sehari yang kadarnya lima pulu

In [22]:
# Generate recommendations for the entire dataset
recomm_list = []
for i in range(len(data)):
    recomm_i = give_recommendations(i)
    recomm_list.append(recomm_i['transformer']['Ayat'])

<ipython-input-12-1c63bd330337>:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  results = data[data['terjemah'].str.contains(query, case=False, na=False)]


error: unbalanced parenthesis at position 545

In [23]:
# Membuat DataFrame dari rekomendasi
recomm_data = pd.DataFrame(recomm_list, columns=['First Recommendation', 'Second Recommendation', 'Third Recommendation', 'Fourth Recommendation', 'Fifth Recommendation'])
recomm_data['Preferred Ayat'] = data['terjemah']
recomm_data = recomm_data[['Preferred Ayat', 'First Recommendation', 'Second Recommendation', 'Third Recommendation', 'Fourth Recommendation', 'Fifth Recommendation']]

In [24]:
# Mengacak dan menampilkan sampel data rekomendasi
recomm_data_shuffled = recomm_data.sample(frac=1).reset_index(drop=True)
print(recomm_data_shuffled.head())

                                      Preferred Ayat  \
0  Kemudian kamu (Bani Israel) membunuh dirimu (s...   
1  Kitab (Al Qur'an) ini tidak ada keraguan padan...   
2  Dan sesungguhnya jika kamu mendatangkan kepada...   
3  kecuali mereka yang telah tobat dan mengadakan...   
4  Dan sungguh akan Kami berikan cobaan kepadamu ...   

                                First Recommendation  \
0  Mereka bertanya kepadamu tentang berperang pad...   
1  (Al Qur'an) ini adalah penerangan bagi seluruh...   
2  Katakanlah: Hai Ahli Kitab kamu tidak dipandan...   
3  kecuali orang-orang yang tobat sesudah (kafir)...   
4  Dan tatkala dia cukup dewasa Kami berikan kepa...   

                               Second Recommendation  \
0  Merekalah orang-orang yang kafir yang menghala...   
1  Al Qur'an ini adalah pedoman bagi manusia petu...   
2  Kemudian jika mereka mendebat kamu (tentang ke...   
3  Maka barang siapa bertobat (di antara pencuri-...   
4  sebagai nikmat dari Kami. Demikianlah Kami 